<a href="https://colab.research.google.com/github/nibaskumar93n-debug/Morphoinformatics/blob/main/Subtractive_genomic_analysis_was_applied_to_the_f_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Performing the **entire subtractive genomic analysis pipeline** as described—from protein sequence retrieval through essentiality and metabolic pathway analysis to subcellular localization—**is generally not feasible to execute *entirely* within a Google Colab notebook using the *exact* external web servers mentioned (UniProt, CD-HIT, BLASTp via NCBI web interface, Geptop, KAAS)**.

However, **it is absolutely possible to replicate the *steps* and perform *equivalent analyses* using Python libraries and command-line tools that can be installed or run within the Colab environment**, though this requires significant coding and setup.

Here is a guide outlining the feasibility and detailing the steps for a **Colab-adapted implementation**.

-----

## 1\. Feasibility of Colab Implementation

| Step | Original Tool | Feasibility in Colab | Notes on Colab Implementation |
| :--- | :--- | :--- | :--- |
| **Protein Retrieval** | UniProt Database | **High** | Use **BioPython** to fetch sequences using accession IDs or use UniProt's API. |
| **Paralog Discarding** | CD-HIT Server | **High** | Install and run **CD-HIT** (command-line version) in Colab's terminal, or use a Python wrapper if available, or write a custom clustering script using a library like `scikit-learn` or `MMseqs2`. |
| **Non-Homologous Identification** | BLASTp (NCBI Web) | **Medium/High** | Use **standalone BLAST+** (easily installed in Colab) and the **BioPython** `NcbiWWW` or `NcbiDblocal` modules. Requires downloading a human proteome database. **This is the most computationally intensive step.** |
| **Essentiality Assessment** | Geptop Server | **Low** | Geptop is a proprietary web server. **Cannot be run directly.** You'd need to find a similar **essential gene prediction tool** (e.g., using machine learning models or comparative genomics data) or use a dataset of known essential genes if available. This step is the **hardest to replicate precisely.** |
| **Metabolic Pathway Analysis** | KAAS Server | **Low** | KAAS is a specialized web server. **Cannot be run directly.** You would use **BioPython** and the **KEGG REST API** (or similar tools like **GhostKOALA** if they offer an API/standalone version) to assign KOs and map to pathways. This requires careful parsing of results. |
| **Subcellular Localization** | (Tool not specified) | **Medium** | Use publicly available **standalone localization prediction tools** like **PSORTb** or **DeepTMHMM** (if available for install) or use an **API** from a service like **DeepLoc** (if one exists). |

-----

## 2\. Step-by-Step Guide for Colab-Adapted Subtractive Genomic Analysis

### A. Setup and Dependencies

The first cell in your Colab notebook will be for installation.

In [1]:
!pip install -q biopython pandas requests
!mkdir -p /content/{proteome,non_paralogous,blast_results}
import requests, os, pandas as pd
from Bio import SeqIO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 31.5 MB/s eta 0:00:00


### B. Protein Sequence Retrieval (UniProt)

Use **BioPython** to fetch the sequences for your four organisms.

In [2]:
# --- STEP 1: Upload your proteome ---
species_name = "Bacteroides_uniformis"
uploaded_proteome = "/content/proteome/Bacteroides_uniformis.fasta"

if os.path.exists(uploaded_proteome):
    os.rename(uploaded_proteome, f"/content/proteome/{species_name}.fasta")
    proteome_path = f"/content/proteome/{species_name}.fasta"
    print(f"✅ Proteome uploaded: {proteome_path}")
else:
    raise FileNotFoundError("❌ Please upload your FASTA file manually in Colab first!")

# --- Count total proteins ---
total_proteins = sum(1 for _ in SeqIO.parse(proteome_path, "fasta"))
print(f"🧩 Total proteins in proteome: {total_proteins}")


✅ Proteome uploaded: /content/proteome/Bacteroides_uniformis.fasta
🧩 Total proteins in proteome: 4618


### C. Paralog Discarding (CD-HIT)

Run the **CD-HIT** command-line tool within Colab using the `!` prefix.

In [3]:
# --- STEP 2. Remove paralogous sequences using CD-HIT (60% identity)
!apt-get install -y cd-hit

# --- STEP 2: Remove paralogs using CD-HIT (60% identity) ---
non_paralog_path = f"/content/non_paralogous/{species_name}_nonparalog.fasta"
os.makedirs("/content/non_paralogous", exist_ok=True)
!cd-hit -i "$proteome_path" -o "$non_paralog_path" -c 0.6 -n 4 -d 0 > /dev/null

# --- Count after CD-HIT ---
non_paralog_count = sum(1 for _ in SeqIO.parse(non_paralog_path, "fasta"))
print(f"🧬 Non-paralogous proteins retained: {non_paralog_count} ({(non_paralog_count/total_proteins)*100:.1f}% retained)")


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  cd-hit
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 521 kB of archives.
After this operation, 1,082 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 cd-hit amd64 4.8.1-4 [521 kB]
Fetched 521 kB in 0s (2,452 kB/s)
Selecting previously unselected package cd-hit.
(Reading database ... 125082 files and directories currently installed.)
Preparing to unpack .../cd-hit_4.8.1-4_amd64.deb ...
Unpacking cd-hit (4.8.1-4) ...
Setting up cd-hit (4.8.1-4) ...
Processing triggers for man-db (2.10.2-1) ...
🧬 Non-paralogous proteins retained: 4419 (95.7% retained)


In [4]:
# --- STEP 3: Remove human homologs ---

# 3a. Install BLAST+
!apt-get install -y ncbi-blast+ > /dev/null
# 3a. Download human reference proteome (UniProt)
!wget -q -O /content/human.fasta "https://rest.uniprot.org/uniprotkb/stream?query=proteome:UP000005640&format=fasta"

# 3b. Build human BLAST database
!makeblastdb -in /content/human.fasta -dbtype prot -out /content/human_db > /dev/null

# 3c. Run BLASTp vs human
blast_out = f"/content/blast_results/{species_name}_vs_human.tsv"
os.makedirs("/content/blast_results", exist_ok=True)
!blastp -query "$non_paralog_path" -db /content/human_db -outfmt "6 qseqid sseqid pident evalue qcovs" -evalue 1e-5 -num_threads 2 -out "$blast_out"

print("✅ BLASTp vs Human completed.")

# --- 3d. Filter for non-homologous proteins (≤30% identity, ≥70% coverage) ---
df_human = pd.read_csv(blast_out, sep="\t", names=["qseqid","sseqid","pident","evalue","qcovs"])
human_homologs = set(df_human[(df_human["pident"] > 30) & (df_human["qcovs"] >= 70)]["qseqid"])
non_homologous_ids = []

for record in SeqIO.parse(non_paralog_path, "fasta"):
    if record.id not in human_homologs:
        non_homologous_ids.append(record.id)

print(f"🚫 Human-homologous proteins removed: {len(human_homologs)}")
print(f"✅ Non-homologous proteins retained: {len(non_homologous_ids)} ({(len(non_homologous_ids)/non_paralog_count)*100:.1f}% retained)")

# --- Save non-homologous FASTA ---
non_hom_fasta = f"/content/{species_name}_nonhomolog.fasta"
with open(non_hom_fasta, "w") as out:
    for record in SeqIO.parse(non_paralog_path, "fasta"):
        if record.id in non_homologous_ids:
            SeqIO.write(record, out, "fasta")

✅ BLASTp vs Human completed.
🚫 Human-homologous proteins removed: 335
✅ Non-homologous proteins retained: 4084 (92.4% retained)


In [5]:
# Unzip DEG10
!gunzip -c /content/DEG10.aa.gz > /content/DEG10.aa.fasta


In [6]:
# --- STEP 4: Predict essential proteins using DEG10 ---
!makeblastdb -in /content/DEG10.aa.fasta -dbtype prot -out /content/deg10_db > /dev/null

blast_deg_out = f"/content/{species_name}_vs_deg10.tsv"
!blastp -query "$non_hom_fasta" -db /content/deg10_db -outfmt "6 qseqid sseqid pident evalue qcovs bitscore" -evalue 1e-5 -num_threads 2 -out "$blast_deg_out"

print("✅ BLASTp vs DEG10 completed.")


FASTA-Reader: Ignoring invalid residues at position(s): On line 91713: 44
FASTA-Reader: Ignoring invalid residues at position(s): On line 102730: 48
FASTA-Reader: Ignoring invalid residues at position(s): On line 110967: 18
FASTA-Reader: Ignoring invalid residues at position(s): On line 112557: 18
FASTA-Reader: Ignoring invalid residues at position(s): On line 112604: 18
FASTA-Reader: Ignoring invalid residues at position(s): On line 112775: 18
FASTA-Reader: Ignoring invalid residues at position(s): On line 113161: 18
FASTA-Reader: Ignoring invalid residues at position(s): On line 113389: 18
FASTA-Reader: Ignoring invalid residues at position(s): On line 113405: 18
FASTA-Reader: Ignoring invalid residues at position(s): On line 113418: 18
FASTA-Reader: Ignoring invalid residues at position(s): On line 113681: 18
FASTA-Reader: Ignoring invalid residues at position(s): On line 113850: 18
FASTA-Reader: Ignoring invalid residues at position(s): On line 114182: 18
FASTA-Reader: Ignoring inv

In [7]:
# --- STEP 5: Filter essential-like hits (GEPTOP mimic logic) ---
df_deg = pd.read_csv(blast_deg_out, sep="\t", names=["qseqid","sseqid","pident","evalue","qcovs","bitscore"])
filtered = df_deg[(df_deg["pident"] >= 30) & (df_deg["qcovs"] >= 70)]

# Keep only best hit per query
best_hits = filtered.sort_values("evalue").drop_duplicates("qseqid", keep="first")

print(f"⭐ Total DEG10 hits passing threshold: {len(filtered)}")
print(f"🎯 Unique predicted essential proteins: {len(best_hits)} ({(len(best_hits)/len(non_homologous_ids))*100:.1f}% of non-homologous proteins)")



⭐ Total DEG10 hits passing threshold: 8676
🎯 Unique predicted essential proteins: 1014 (24.8% of non-homologous proteins)


In [8]:
 # --- STEP 5: Filter essential-like hits (stricter GEPTOP mimic logic) ---Getting too many essential proteins so using stricter threshold
df_deg = pd.read_csv(blast_deg_out, sep="\t", names=["qseqid","sseqid","pident","evalue","qcovs","bitscore"])

# ✅ Tightened thresholds for higher confidence
filtered = df_deg[
    (df_deg["pident"] >= 40) &
    (df_deg["qcovs"] >= 80) &
    (df_deg["bitscore"] >= 100) &
    (df_deg["evalue"] <= 1e-10)
]

# Keep only the best hit per protein
best_hits = filtered.sort_values("evalue").drop_duplicates("qseqid", keep="first")

print(f"⭐ Total DEG10 hits passing strict threshold: {len(filtered)}")
print(f"🎯 Unique predicted essential proteins: {len(best_hits)} ({(len(best_hits)/len(non_homologous_ids))*100:.1f}% of non-homologous proteins)")


⭐ Total DEG10 hits passing strict threshold: 3431
🎯 Unique predicted essential proteins: 607 (14.9% of non-homologous proteins)


In [9]:
# --- STEP 6: Extract FASTA for essential proteins ---
ids_to_keep = set(best_hits["qseqid"])
output_fasta = f"/content/{species_name}_predicted_essential.fasta"

with open(output_fasta, "w") as out:
    for record in SeqIO.parse(non_hom_fasta, "fasta"):
        if record.id in ids_to_keep:
            SeqIO.write(record, out, "fasta")

print(f"💾 FASTA saved: {output_fasta}")


💾 FASTA saved: /content/Bacteroides_uniformis_predicted_essential.fasta


In [10]:
# --- STEP 6: Extract FASTA for essential proteins ---
ids_to_keep = set(best_hits["qseqid"])
output_fasta = f"/content/{species_name}_predicted_essential_revised_threshold.fasta"

with open(output_fasta, "w") as out:
    for record in SeqIO.parse(non_hom_fasta, "fasta"):
        if record.id in ids_to_keep:
            SeqIO.write(record, out, "fasta")

print(f"💾 FASTA saved: {output_fasta}")

💾 FASTA saved: /content/Bacteroides_uniformis_predicted_essential_revised_threshold.fasta


In [11]:
import pandas as pd
import requests
from tqdm import tqdm

# 1️⃣ Load KAAS mapping (protein → KO)
kaas_file = "/content/Kaas_bacteroides_uniformis.csv"  # your CSV file

# Read as standard CSV (comma-separated)
df = pd.read_csv(kaas_file)

# Check column names
print("Columns detected:", df.columns.tolist())
if not {"protein", "KO"}.issubset(df.columns):
    df.columns = ["protein", "KO"]  # enforce standard naming if not present

# 2️⃣ Count assigned and unassigned
assigned = df["KO"].notna().sum()
unassigned = df["KO"].isna().sum()
print(f"Assigned KO IDs: {assigned}")
print(f"Unassigned proteins: {unassigned}")

# 3️⃣ Remove NA and get unique KO IDs
ko_list = df["KO"].dropna().unique().tolist()

# 4️⃣ Map each KO to KEGG pathways via KEGG REST API
def get_pathways_for_ko(ko):
    url = f"https://rest.kegg.jp/link/pathway/ko:{ko}"
    res = requests.get(url)
    if res.status_code == 200:
        lines = res.text.strip().split("\n")
        pathways = []
        for l in lines:
            parts = l.split("\t")
            if len(parts) > 1:  # only if both columns exist
                pathways.append(parts[1].replace("path:", ""))
        return pathways
    return []


ko_to_path = {}
for ko in tqdm(ko_list, desc="Mapping KO → Pathway"):
    ko_to_path[ko] = get_pathways_for_ko(ko)

# 5️⃣ Create DataFrame of KO → Pathway
path_df = (
    pd.DataFrame([(ko, p) for ko, plist in ko_to_path.items() for p in plist],
                 columns=["KO", "Pathway"])
)

# 6️⃣ Identify KO IDs with no pathway mapping
mapped_kos = set(path_df["KO"])
unmapped_kos = [ko for ko in ko_list if ko not in mapped_kos]
print(f"\nKO-assigned proteins with NO pathway mapping: {len(unmapped_kos)}")

# 7️⃣ Download human pathway list
human_pathways = requests.get("http://rest.kegg.jp/list/pathway/hsa").text
human_path_list = [line.split("\t")[0].replace("path:", "") for line in human_pathways.strip().split("\n")]

# 8️⃣ Identify shared vs unique bacterial pathways
path_df["Shared_with_Human"] = path_df["Pathway"].isin(human_path_list)

shared = path_df[path_df["Shared_with_Human"]].Pathway.nunique()
unique = path_df[~path_df["Shared_with_Human"]].Pathway.nunique()

print(f"\n🧭 Pathway summary:")
print(f"Total distinct pathways: {path_df.Pathway.nunique()}")
print(f"Shared with Human: {shared}")
print(f"Unique bacterial: {unique}")

# 9️⃣ Save results
path_df.to_csv("/content/KAAS_pathway_analysis.csv", index=False)
print("\n✅ Results saved to: /content/KAAS_pathway_analysis.csv")

# 🔟 Save unique bacterial pathways only
unique_df = path_df[~path_df["Shared_with_Human"]]
unique_df.to_csv("/content/unique_bacterial_pathways.csv", index=False)
print("🧬 Unique bacterial pathways saved: /content/unique_bacterial_pathways.csv")



Columns detected: ['Protein', 'KO']
Assigned KO IDs: 415
Unassigned proteins: 192


Mapping KO → Pathway: 100%|██████████| 402/402 [04:59<00:00,  1.34it/s]



KO-assigned proteins with NO pathway mapping: 118

🧭 Pathway summary:
Total distinct pathways: 222
Shared with Human: 0
Unique bacterial: 222

✅ Results saved to: /content/KAAS_pathway_analysis.csv
🧬 Unique bacterial pathways saved: /content/unique_bacterial_pathways.csv


In [12]:
import pandas as pd

# --- Input files ---
kaas_file = "/content/Kaas_bacteroides_uniformis.csv"      # Protein → KO mapping (comma-delimited)
pathway_file = "/content/KAAS_pathway_analysis.csv"      # KO → Pathway mapping (comma-delimited)

# --- Load data ---
mapping_df = pd.read_csv(kaas_file)  # uses header from file
path_df = pd.read_csv(pathway_file)

# --- Summary: Assigned vs Unassigned KOs ---
assigned = mapping_df["KO"].notna().sum()
unassigned = mapping_df["KO"].isna().sum()

# --- KO → pathway mapping ---
ko_list = mapping_df["KO"].dropna().unique().tolist()
ko_with_no_pathway = [ko for ko in ko_list if ko not in path_df["KO"].unique()]
num_no_pathway = len(ko_with_no_pathway)

# --- Filter only unique bacterial pathways ---
unique_pathways_df = path_df[path_df["Shared_with_Human"] == False]

# --- Merge protein → KO with KO → pathway ---
merged_df = pd.merge(mapping_df.dropna(subset=["KO"]), unique_pathways_df, on="KO", how="inner")
merged_df = merged_df.drop_duplicates(subset=["Protein", "KO", "Pathway"])

# --- Save merged protein → KO → pathway CSV ---
output_file = "/content/Bacteroides_uniformis_merged_information.csv"
merged_df.to_csv(output_file, index=False)

# --- Save summary info ---
summary_file = "/content/Bacteroides_uniformis_KO_summary.csv"
summary_df = pd.DataFrame({
    "Metric": ["Assigned KO IDs", "Unassigned proteins", "KO-assigned proteins with NO pathway mapping",
               "Total distinct pathways", "Shared with Human", "Unique bacterial pathways"],
    "Count": [assigned, unassigned, num_no_pathway,
              path_df["Pathway"].nunique(), path_df[path_df["Shared_with_Human"]].Pathway.nunique(),
              unique_pathways_df.Pathway.nunique()]
})
summary_df.to_csv(summary_file, index=False)

# --- Print info ---
print(f"✅ Merged protein → KO → pathway file saved: {output_file}")
print(f"✅ Summary file saved: {summary_file}")
print(summary_df)


✅ Merged protein → KO → pathway file saved: /content/Bacteroides_uniformis_merged_information.csv
✅ Summary file saved: /content/Bacteroides_uniformis_KO_summary.csv
                                         Metric  Count
0                               Assigned KO IDs    415
1                           Unassigned proteins    192
2  KO-assigned proteins with NO pathway mapping    118
3                       Total distinct pathways    222
4                             Shared with Human      0
5                     Unique bacterial pathways    222


In [13]:
import pandas as pd

# --- Load KAAS results (protein ↔ KO) ---
kaas_df = pd.read_csv("/content/Kaas_bacteroides_uniformis.csv")

# --- Load KO ↔ Pathway data (from your previous analysis) ---
path_df = pd.read_csv("/content/KAAS_pathway_analysis.csv")

# --- Filter for unique bacterial pathways ---
unique_df = path_df[path_df["Shared_with_Human"] == False]

# --- Get list of unique KO IDs ---
unique_kos = unique_df["KO"].unique()

# --- Subset proteins belonging to those KOs ---
unique_proteins = kaas_df[kaas_df["KO"].isin(unique_kos)]

# --- Save list of unique proteins ---
unique_proteins.to_csv("/content/Bacteroides_uniformis_unique_pathway_proteins.csv", index=False)
print(f"✅ Unique proteins saved: {unique_proteins.shape[0]}")


✅ Unique proteins saved: 291


In [15]:
from Bio import SeqIO
import pandas as pd

# --- INPUT FILES ---
fasta_file = "/content/Bacteroides_uniformis_predicted_essential_revised_threshold.fasta"
pathway_file = "/content/KAAS_pathway_analysis.csv"
mapping_file = "/content/Kaas_bacteroides_uniformis.csv"

# --- LOAD DATA ---
path_df = pd.read_csv(pathway_file)
mapping_df = pd.read_csv(mapping_file, sep=",")  # 👈 changed from '\t' to ','

# Ensure column names are correct
mapping_df.columns = ["protein", "KO"]

# Get KOs that are NOT shared with human (unique bacterial)
unique_kos = path_df.loc[path_df["Shared_with_Human"] == False, "KO"].unique().tolist()

# Get protein IDs associated with those unique KOs
unique_proteins = mapping_df[mapping_df["KO"].isin(unique_kos)]["protein"].unique().tolist()

print(f"✅ Unique bacterial KOs: {len(unique_kos)}")
print(f"✅ Corresponding protein IDs: {len(unique_proteins)}")

# --- FILTER FASTA ---
output_fasta = "/content/Bacteroides_uniformis_unique_pathway_proteins.fasta"
count = 0

with open(output_fasta, "w") as out_f:
    for record in SeqIO.parse(fasta_file, "fasta"):
        if any(pid in record.id for pid in unique_proteins):
            SeqIO.write(record, out_f, "fasta")
            count += 1

print(f"🎯 Unique-pathway protein sequences saved: {output_fasta}")
print(f"Total sequences written: {count}")





✅ Unique bacterial KOs: 284
✅ Corresponding protein IDs: 291
🎯 Unique-pathway protein sequences saved: /content/Bacteroides_uniformis_unique_pathway_proteins.fasta
Total sequences written: 291


In [17]:
import pandas as pd
import re

# Path to your PSORTb CSV
psortb_csv = "/content/PSORTb_results.csv"

# Read as plain text (since all info is in one column)
df_raw = pd.read_csv(psortb_csv, header=None, names=["Text"], dtype=str)
print(f"✅ Loaded {len(df_raw)} rows from PSORTb result")



✅ Loaded 7275 rows from PSORTb result


In [19]:
import pandas as pd
import re

# Load your raw PSORTb results (text in one column)
df_raw = pd.read_csv("/content/PSORTb_results.csv")  # change filename if needed

# Join all rows into one large text block
content = "\n".join(df_raw.iloc[:,0].tolist())

# Split by "SeqID:"
entries = re.split(r"SeqID:", content)
records = []

for entry in entries:
    entry = entry.strip()
    if not entry:
        continue

    # Extract the sequence ID
    seq_match = re.search(r"^\s*(\S+)", entry)

    # Extract the final prediction localization
    loc_match = re.search(r"Final Prediction:\s*(\w+)", entry)

    if seq_match and loc_match:
        seqid = seq_match.group(1).strip()
        loc = loc_match.group(1).strip()
        records.append((seqid, loc))

# Create a dataframe
df = pd.DataFrame(records, columns=["SeqID", "Localization"])

# Save the cleaned file
df.to_csv("/content/psortb_cleaned.csv", index=False)

print(f"✅ Extracted {len(df)} protein predictions")
print("💾 Saved as: /content/psortb_cleaned.csv")
df.head()



TypeError: sequence item 21: expected str instance, float found

In [ ]:
import pandas as pd

# Load your PSORTb results
df = pd.read_csv("/content/psortb_cleaned.csv")

# Normalize localization text to lowercase and strip whitespace
df["Localization"] = df["Localization"].str.lower().str.strip()

# 🧪 Cytoplasmic ONLY (exclude cytoplasmic membrane)
cytoplasmic_only = df[df["Localization"] == "cytoplasmic"]

# 🧬 Cytoplasmic Membrane
cytoplasmic_membrane = df[
    df["Localization"].str.contains(
        "cytoplasmicmembrane|cytoplasmic membrane",
        case=False,
        na=False
    )
]

# 💉 Vaccine candidates (outer membrane, extracellular, periplasmic)
vaccine_candidates = df[
    df["Localization"].str.contains(
        "outermembrane|outer membrane|extracellular|periplasm",
        case=False,
        na=False
    )
]

# ❓ Unknown localization
unknown_localization = df[df["Localization"] == "unknown"]

# 📊 Print comprehensive summary
print(f"\n{'='*60}")
print(f"📊 SUBCELLULAR LOCALIZATION ANALYSIS")
print(f"{'='*60}")
print(f"Total proteins analyzed: {len(df)}")
print(f"\n🔬 Localization Distribution:")
print(f"  🧪 Cytoplasmic only: {len(cytoplasmic_only)} ({len(cytoplasmic_only)/len(df)*100:.1f}%)")
print(f"  🧬 Cytoplasmic membrane: {len(cytoplasmic_membrane)} ({len(cytoplasmic_membrane)/len(df)*100:.1f}%)")
print(f"  💉 Outer membrane/Extracellular/Periplasmic: {len(vaccine_candidates)} ({len(vaccine_candidates)/len(df)*100:.1f}%)")
print(f"  ❓ Unknown: {len(unknown_localization)} ({len(unknown_localization)/len(df)*100:.1f}%)")
print(f"{'='*60}\n")

# 🎯 Drug target prioritization (following the paper's approach - keep all but categorize)
print(f"🎯 DRUG TARGET PRIORITIZATION:")
print(f"  ⭐ High priority (membrane/secreted): {len(vaccine_candidates)} proteins")
print(f"     → Accessible, good for antibodies/small molecules")
print(f"  ⭐ Medium priority (cytoplasmic membrane): {len(cytoplasmic_membrane)} proteins")
print(f"     → Targetable with membrane-permeable drugs")
print(f"  ⭐ Lower priority (cytoplasmic): {len(cytoplasmic_only)} proteins")
print(f"     → Requires cell penetration, but still valid targets")
print(f"  ⚠️  Unknown localization: {len(unknown_localization)} proteins")
print(f"     → Needs further analysis\n")

# Save all categories
df.to_csv("/content/psortb_cleaned.csv", index=False)
cytoplasmic_only.to_csv("/content/cytoplasmic_only.csv", index=False)
cytoplasmic_membrane.to_csv("/content/cytoplasmic_membrane.csv", index=False)
vaccine_candidates.to_csv("/content/vaccine_candidates.csv", index=False)
unknown_localization.to_csv("/content/unknown_localization.csv", index=False)

# Combined drug targets (all except pure cytoplasmic if you want to be strict)
# But following the paper - keep ALL
all_targets = df.copy()
all_targets.to_csv("/content/all_localized_targets.csv", index=False)

print("✅ Files saved:")
print("  - psortb_cleaned.csv (all proteins)")
print("  - cytoplasmic_only.csv")
print("  - cytoplasmic_membrane.csv")
print("  - vaccine_candidates.csv")
print("  - unknown_localization.csv")
print("  - all_localized_targets.csv")

# 📈 Detailed breakdown
print(f"\n📈 Detailed Localization Breakdown:")
print(df["Localization"].value_counts())








📊 SUBCELLULAR LOCALIZATION ANALYSIS
Total proteins analyzed: 295

🔬 Localization Distribution:
  🧪 Cytoplasmic only: 238 (80.7%)
  🧬 Cytoplasmic membrane: 37 (12.5%)
  💉 Outer membrane/Extracellular/Periplasmic: 4 (1.4%)
  ❓ Unknown: 16 (5.4%)

🎯 DRUG TARGET PRIORITIZATION:
  ⭐ High priority (membrane/secreted): 4 proteins
     → Accessible, good for antibodies/small molecules
  ⭐ Medium priority (cytoplasmic membrane): 37 proteins
     → Targetable with membrane-permeable drugs
  ⭐ Lower priority (cytoplasmic): 238 proteins
     → Requires cell penetration, but still valid targets
  ⚠️  Unknown localization: 16 proteins
     → Needs further analysis

✅ Files saved:
  - psortb_cleaned.csv (all proteins)
  - cytoplasmic_only.csv
  - cytoplasmic_membrane.csv
  - vaccine_candidates.csv
  - unknown_localization.csv
  - all_localized_targets.csv

📈 Detailed Localization Breakdown:
Localization
cytoplasmic            238
cytoplasmicmembrane     37
unknown                 16
periplasmic     

In [ ]:
### For Gene_symbol from Uniprot

ids = df["SeqID"].dropna().unique().tolist()
print(f"Found {len(ids)} unique protein IDs")


Found 295 unique protein IDs


In [ ]:
import requests
import time
import pandas as pd

def get_gene_name_from_uniprot(SeqID):
    """Fetch gene name from UniProt API"""
    try:
        # Clean protein ID (remove version numbers like .1, .2)
        clean_id = SeqID.split('.')[0].split('|')[-1]

        url = f"https://rest.uniprot.org/uniprotkb/{clean_id}.json"
        response = requests.get(url, timeout=10)

        if response.status_code == 200:
            data = response.json()
            # Try to get gene name
            if 'genes' in data and len(data['genes']) > 0:
                if 'geneName' in data['genes'][0]:
                    return data['genes'][0]['geneName']['value']

        return None
    except:
        return None

# Load cytoplasmic proteins
df_cytoplasmic = pd.read_csv("/content/cytoplasmic_only.csv")

# Get gene names (this will take time - ~234 API calls)
gene_names = []
print("Fetching gene names from UniProt...")

for i, SeqID in enumerate(df_cytoplasmic['SeqID']):
    gene_name = get_gene_name_from_uniprot(SeqID)
    gene_names.append(gene_name if gene_name else SeqID)

    if (i + 1) % 10 == 0:
        print(f"  Processed {i + 1}/{len(df_cytoplasmic)} proteins...")
        time.sleep(0.5)  # Be nice to UniProt API

df_cytoplasmic['Gene_Name'] = gene_names
df_cytoplasmic.to_csv("/content/cytoplasmic_with_genes.csv", index=False)
print("✅ Gene names retrieved and saved!")


Fetching gene names from UniProt...
  Processed 10/238 proteins...
  Processed 20/238 proteins...
  Processed 30/238 proteins...
  Processed 40/238 proteins...
  Processed 50/238 proteins...
  Processed 60/238 proteins...
  Processed 70/238 proteins...
  Processed 80/238 proteins...
  Processed 90/238 proteins...
  Processed 100/238 proteins...
  Processed 110/238 proteins...
  Processed 120/238 proteins...
  Processed 130/238 proteins...
  Processed 140/238 proteins...
  Processed 150/238 proteins...
  Processed 160/238 proteins...
  Processed 170/238 proteins...
  Processed 180/238 proteins...
  Processed 190/238 proteins...
  Processed 200/238 proteins...
  Processed 210/238 proteins...
  Processed 220/238 proteins...
  Processed 230/238 proteins...
✅ Gene names retrieved and saved!


In [ ]:
### using batch  to find gene symbol import requests### not very handy
import pandas as pd
import time

def batch_uniprot_mapping(SeqID):
    """Map multiple protein IDs to gene names in one request"""

    # Prepare the mapping request
    url = "https://rest.uniprot.org/idmapping/run"

    params = {
        'ids': ','.join(SeqID[:500]),  # Max 500 at a time
        'from': 'UniProtKB_AC-ID',
        'to': 'Gene_Name'
    }

    response = requests.post(url, data=params)

    if response.status_code == 200:
        job_id = response.json()['jobId']

        # Poll for results
        results_url = f"https://rest.uniprot.org/idmapping/status/{job_id}"

        for _ in range(30):  # Try for 30 seconds
            time.sleep(1)
            status = requests.get(results_url).json()

            if 'results' in status:
                return status['results']

    return []

# Use this for your 234 proteins
df_cytoplasmic = pd.read_csv("/content/cytoplasmic_only.csv")
SeqID = df_cytoplasmic['SeqID'].tolist()

# Clean IDs
clean_ids = [pid.split('.')[0].split('|')[-1] for pid in SeqID]

results = batch_uniprot_mapping(clean_ids[:500])  # First 500

# Parse results
id_to_gene = {r['from']: r['to'] for r in results}
df_cytoplasmic['Gene_Name'] = df_cytoplasmic['SeqID'].apply(
    lambda x: id_to_gene.get(x.split('.')[0].split('|')[-1], x)
)

# 💾 Save output file
output_path = "/content/cytoplasmic_with_genes_batch.csv"
df_cytoplasmic.to_csv(output_path, index=False)
print(f"✅ Output file saved to: {output_path}")



✅ Output file saved to: /content/cytoplasmic_with_genes_batch.csv


In [ ]:
from Bio import SeqIO

input_fasta = "/content/Bifidobacterium_animalis_unique_pathway_proteins.fasta"
records = list(SeqIO.parse(input_fasta, "fasta"))

chunk_size = 70
for i in range(0, len(records), chunk_size):
    chunk = records[i:i+chunk_size]
    output_file = f"/content/unique_proteins_chunk_{i//chunk_size + 1}.fasta"
    SeqIO.write(chunk, output_file, "fasta")
    print(f"✅ Chunk saved: {output_file} ({len(chunk)} sequences)")


✅ Chunk saved: /content/unique_proteins_chunk_1.fasta (70 sequences)
✅ Chunk saved: /content/unique_proteins_chunk_2.fasta (60 sequences)


In [ ]:
##### The NetGenes databse system is not clear to me now ,,, because there is no protein sequence onley the essential gene and their scores........
from google.colab import files

uploaded = files.upload()  # Upload your NetGenes zip file





Saving NetGenes.zip to NetGenes.zip
